# Object Detection

## Loading Data

In [1]:
import turicreate as tc

In [2]:
import os

In [9]:
data_path = 'data'

**Load all images in random order**

In [10]:
raw_sf = tc.image_analysis.load_images(data_path, recursive=True, random_order=True)

Unsupported image format. Supported formats are JPEG and PNG	 file: data/people/people_suitability.txt

Unsupported image format. Supported formats are JPEG and PNG	 file: data/bikes/bikes_suitability.txt

Unsupported image format. Supported formats are JPEG and PNG	 file: data/cars/cars_suitability.txt

| path	| image |

[3162 rows x 2 columns]

**Split file names according to the folder**

In [17]:
info = raw_sf['path'].apply(lambda path: os.path.basename(path).split('.')[:2])

**Rename columns to 'name' and 'type'**

In [18]:
info = info.unpack().rename({'X.0': 'name', 'X.1': 'type'})

In [15]:
info

name,type
bike_351,mask
bike_086,mask
carsgraz_027,mask
carsgraz_107,mask
person_179,image
bike_264,mask
bike_187,mask
carsgraz_233,mask
carsgraz_300,mask
carsgraz_183,mask


In [19]:
raw_sf = raw_sf.add_columns(info)

In [24]:
raw_sf

image,name,type,label
Height: 480 Width: 640,bike_351,mask,bike
Height: 480 Width: 640,bike_086,mask,bike
Height: 480 Width: 640,carsgraz_027,mask,carsgraz
Height: 480 Width: 640,carsgraz_107,mask,carsgraz
Height: 480 Width: 640,person_179,image,person
Height: 480 Width: 640,bike_264,mask,bike
Height: 480 Width: 640,bike_187,mask,bike
Height: 480 Width: 640,carsgraz_233,mask,carsgraz
Height: 480 Width: 640,carsgraz_300,mask,carsgraz
Height: 480 Width: 640,carsgraz_183,mask,carsgraz


In [21]:
raw_sf['label'] = raw_sf['name'].apply(lambda name: name.split('_')[0])

In [23]:
del raw_sf['path']

In [25]:
sf_images = raw_sf[raw_sf['type'] == 'image']

[1096 rows x 4 columns]

In [27]:
sf_masks = raw_sf[raw_sf['type'] == 'mask']

[2066 rows x 4 columns]

**Define a function for helping finding the mask coordinate**
* Input: tc.Image of a mask
* Return: bounding box coordinates dictionary e.g. {'x': 100, 'y': 120, 'width': 80, 'height': 120}

In [29]:
def mask_to_bbox_coordinates(img):
    import numpy as np
    mask = img.pixel_data
    if mask.max() == 0:
        return None
    
    x0, x1 = np.where(mask.max(0))[0][[0, -1]]
    y0, y1 = np.where(mask.max(1))[1][[0, -1]]
    
    return {
        'x': (x0 + x1) / 2, 'width': (x1 - x0),
        'y': (y0 + y1) / 2, 'height': (y1 - y0)
    }

**Convert masks to bounding box(drop masks that did not contain bounding box)**

In [34]:
sf_masks['coordinates'] = sf_masks['image'].apply(mask_to_bbox_coordinates)

In [37]:
sf_masks

image,name,type,bbox
Height: 480 Width: 640,bike_351,mask,"{'label': 'bike','coordinates': {'x': ..."
Height: 480 Width: 640,bike_086,mask,"{'label': 'bike','coordinates': {'x': ..."
Height: 480 Width: 640,carsgraz_027,mask,"{'label': 'carsgraz','coordinates': {'x': ..."
Height: 480 Width: 640,carsgraz_107,mask,"{'label': 'carsgraz','coordinates': {'x': ..."
Height: 480 Width: 640,bike_264,mask,"{'label': 'bike','coordinates': {'x': ..."
Height: 480 Width: 640,bike_187,mask,"{'label': 'bike','coordinates': {'x': ..."
Height: 480 Width: 640,carsgraz_233,mask,"{'label': 'carsgraz','coordinates': {'x': ..."
Height: 480 Width: 640,carsgraz_300,mask,"{'label': 'carsgraz','coordinates': {'x': ..."
Height: 480 Width: 640,carsgraz_183,mask,"{'label': 'carsgraz','coordinates': {'x': ..."
Height: 480 Width: 640,bike_079,mask,"{'label': 'bike','coordinates': {'x': ..."


**Drop empty masks(which returns None)**

In [32]:
sf_masks = sf_masks.dropna('coordinates')

In [36]:
sf_masks = sf_masks.pack_columns(['label', 'coordinates'], new_column_name='bbox', dtype=dict)

In [38]:
sf_annotations = sf_masks.groupby('name', 
                                  {'annotations': tc.aggregate.CONCAT('bbox')})

**Combine bounding boxes of the same 'name' into lists**

In [39]:
sf_annotations

name,annotations
person_116,"[{'label': 'person','coordinates': {'x': ..."
carsgraz_120,"[{'label': 'carsgraz','coordinates': {'x': ..."
person_199,"[{'label': 'person','coordinates': {'x': ..."
bike_137,"[{'label': 'bike','coordinates': {'x': ..."
carsgraz_045,"[{'label': 'carsgraz','coordinates': {'x': ..."
person_037,"[{'label': 'person','coordinates': {'x': ..."
bike_078,"[{'label': 'bike','coordinates': {'x': ..."
person_184,"[{'label': 'person','coordinates': {'x': ..."
carsgraz_293,"[{'label': 'carsgraz','coordinates': {'x': ..."
carsgraz_255,"[{'label': 'carsgraz','coordinates': {'x': ..."


**Join annotations with images**
* Note: some images do not have annotations, but we still want to keep them, So, we set a LEFT join

In [40]:
sf = sf_images.join(sf_annotations, on='name', how='left')

In [44]:
sf

image,name,label,annotations
Height: 480 Width: 640,person_179,person,"[{'label': 'person','coordinates': {'x': ..."
Height: 480 Width: 640,carsgraz_137,carsgraz,"[{'label': 'carsgraz','coordinates': {'x': ..."
Height: 480 Width: 640,carsgraz_228,carsgraz,"[{'label': 'carsgraz','coordinates': {'x': ..."
Height: 480 Width: 640,carsgraz_193,carsgraz,"[{'label': 'carsgraz','coordinates': {'x': ..."
Height: 480 Width: 640,carsgraz_419,carsgraz,"[{'label': 'carsgraz','coordinates': {'x': ..."
Height: 480 Width: 640,bike_033,bike,"[{'label': 'bike','coordinates': {'x': ..."
Height: 480 Width: 640,bike_185,bike,"[{'label': 'bike','coordinates': {'x': ..."
Height: 480 Width: 640,bike_122,bike,"[{'label': 'bike','coordinates': {'x': ..."
Height: 480 Width: 640,carsgraz_120,carsgraz,"[{'label': 'carsgraz','coordinates': {'x': ..."
Height: 480 Width: 640,person_234,person,"[{'label': 'person','coordinates': {'x': ..."


In [42]:
sf['annotations'] = sf['annotations'].fillna([])

In [43]:
del sf['type']

In [45]:
sf.save('objectDetection.sframe')